## Scraper : Amazon Perfume - USA


In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import json
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
import os
from collections import Counter
import pickle
import warnings
import time
warnings.filterwarnings("ignore")

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageFilter
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

from utils import hover, browser, scroll_temp, scroll, browser_link, indexes, products_links, delete_images, upload_s3, product_info, database, checkpoint

import boto3
import botocore
%matplotlib inline

# Use proxy and headers for safe web scraping
# os.environ['HTTPS_PROXY'] = 'http://3.112.188.39:8080'

# pd.options.mode.chained_assignment = None
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/'
    '537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

C:\Users\Dell-pc\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
countries_link = {'USA':'https://www.amazon.com',
                  'Australia':'https://www.amazon.com.au',
                  'UK':'https://www.amazon.co.uk',
                  'India':'https://www.amazon.in',
                  'Japan':'https://www.amazon.co.jp/',
                  'UAE':'https://amazon.ae'}

##### List of Products

In [7]:
amazon_usa = {'health_and_beauty':{'hair_products':{'shampoo':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11057241%2Cn%3A17911764011%2Cn%3A11057651&dc&',
                                                    'conditioner':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11057241%2Cn%3A17911764011%2Cn%3A11057251&dc&',
                                                    'hair_scalp_treatment':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11057241%2Cn%3A11057431&dc&',
                                                    'treatment_oil':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11057241%2Cn%3A10666439011&dc&',
                                                    'hair_loss':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11057241%2Cn%3A10898755011&dc&'},
                                   'skin_care':{'body':{'cleansers':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060521%2Cn%3A11056281&dc&',
                                                        'moisturizers':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060521%2Cn%3A11060661&dc&',
                                                        'treatments':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060521%2Cn%3A11056421&dc&'},
                                                'eyes':{'creams':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11061941%2Cn%3A7730090011&dc&',
                                                        'gels':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11061941%2Cn%3A7730092011&dc&',
                                                        'serums':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11061941%2Cn%3A7730098011&dc&'},
                                                'face':{'f_cleansers':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060711%2Cn%3A11060901&dc&',
                                                        'f_moisturizers':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060711%2Cn%3A11060901&dc&',
                                                        'scrubs':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060711%2Cn%3A11061091&dc&',
                                                        'toners':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060711%2Cn%3A11061931&dc&',
                                                        'f_treatments':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A11060711%2Cn%3A11061931&dc&'},
                                                'lipcare':'https://www.amazon.com/s?i=beauty-intl-ship&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11060451%2Cn%3A3761351&dc&',
                                                'perfumes':'https://www.amazon.com/s?i=specialty-aps&bbn=16225006011&rh=n%3A%2116225006011%2Cn%3A11056591&'}},
              'food':{'tea':{'herbal':'https://www.amazon.com/s?k=tea&i=grocery&rh=n%3A16310101%2Cn%3A16310231%2Cn%3A16521305011%2Cn%3A16318401%2Cn%3A16318511&dc&',
                             'green':'https://www.amazon.com/s?k=tea&i=grocery&rh=n%3A16310101%2Cn%3A16310231%2Cn%3A16521305011%2Cn%3A16318401%2Cn%3A16318471&dc&',
                             'black':'https://www.amazon.com/s?k=tea&i=grocery&rh=n%3A16310101%2Cn%3A16310231%2Cn%3A16521305011%2Cn%3A16318401%2Cn%3A16318411&dc&',
                             'chai':'https://www.amazon.com/s?k=tea&i=grocery&rh=n%3A16310101%2Cn%3A16310231%2Cn%3A16521305011%2Cn%3A16318401%2Cn%3A348022011&dc&'},
                      'coffee':'https://www.amazon.com/s?k=tea&i=grocery&rh=n%3A16310101%2Cn%3A16310231%2Cn%3A16521305011%2Cn%3A16318031%2Cn%3A2251593011&dc&',
                      'dried_fruits':{'mixed':'https://www.amazon.com/s?k=dried+fruits&i=grocery&rh=n%3A16310101%2Cn%3A6506977011%2Cn%3A9865332011%2Cn%3A9865334011%2Cn%3A9865348011&dc&',
                                      'mangoes':'https://www.amazon.com/s?k=dried+fruits&rh=n%3A16310101%2Cn%3A9865346011&dc&'},
                      'nuts':{'mixed':'https://www.amazon.com/s?k=nuts&rh=n%3A16310101%2Cn%3A16322931&dc&',
                              'peanuts':'https://www.amazon.com/s?k=nuts&i=grocery&rh=n%3A16310101%2Cn%3A18787303011%2Cn%3A16310221%2Cn%3A16322881%2Cn%3A16322941&dc&',
                              'cashews':'https://www.amazon.com/s?k=nuts&i=grocery&rh=n%3A16310101%2Cn%3A18787303011%2Cn%3A16310221%2Cn%3A16322881%2Cn%3A16322901&dc&'}},
              'supplements':{'sports':{'pre_workout':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A6973663011%2Cn%3A6973697011&dc&',
                                       'protein':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A6973663011%2Cn%3A6973704011&dc&',
                                       'fat_burner':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A6973663011%2Cn%3A6973679011&dc&',
                                       'weight_gainer':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A6973663011%2Cn%3A6973725011&dc&'},
                             'vitamins_dietary':{'supplements':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A3764441%2Cn%3A6939426011&dc&',
                                                 'multivitamins':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A3774861&dc&'}},
              'wellness':{'ayurveda':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A10079996011%2Cn%3A13052911%2Cn%3A13052941&dc&',
                          'essential_oil_set':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A10079996011%2Cn%3A13052911%2Cn%3A18502613011&dc&',
                          'massage_oil':'https://www.amazon.com/s?k=supplements&i=hpc&rh=n%3A3760901%2Cn%3A10079996011%2Cn%3A14442631&dc&'},
              'personal_accessories':{'bags':{'women':{'clutches':'https://www.amazon.com/s?k=bags&i=fashion-womens-handbags&bbn=15743631&rh=n%3A7141123011%2Cn%3A%217141124011%2Cn%3A7147440011%2Cn%3A15743631%2Cn%3A17037745011&dc&',
                                                       'crossbody':'https://www.amazon.com/s?k=bags&i=fashion-womens-handbags&bbn=15743631&rh=n%3A7141123011%2Cn%3A%217141124011%2Cn%3A7147440011%2Cn%3A15743631%2Cn%3A2475899011&dc&',
                                                       'fashion':'https://www.amazon.com/s?k=bags&i=fashion-womens-handbags&bbn=15743631&rh=n%3A7141123011%2Cn%3A%217141124011%2Cn%3A7147440011%2Cn%3A15743631%2Cn%3A16977745011&dc&',
                                                       'hobo':'https://www.amazon.com/s?k=bags&i=fashion-womens-handbags&bbn=15743631&rh=n%3A7141123011%2Cn%3A%217141124011%2Cn%3A7147440011%2Cn%3A15743631%2Cn%3A16977747011&dc&'}},
                                      'jewelry':{'anklets':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454897011&dc&',
                                                 'bracelets':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454898011&dc&',
                                                 'earrings':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454917011&dc&',
                                                 'necklaces':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454917011&dc&',
                                                 'rings':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454939011&dc&'},
                                      'artisan_fabrics':'https://www.amazon.com/s?k=fabrics&rh=n%3A2617941011%2Cn%3A12899121&dc&'}}
amazon_uk = {'health_and_beauty':{'hair_products':{'shampoo':'https://www.amazon.co.uk/b/ref=amb_link_5?ie=UTF8&node=74094031&pf_rd_m=A3P5ROKL5A1OLE&pf_rd_s=merchandised-search-leftnav&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_t=101&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_i=66469031',
                                                   'conditioner':'https://www.amazon.co.uk/b/ref=amb_link_6?ie=UTF8&node=2867976031&pf_rd_m=A3P5ROKL5A1OLE&pf_rd_s=merchandised-search-leftnav&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_t=101&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_i=66469031',
                                                   'hair_loss':'https://www.amazon.co.uk/b/ref=amb_link_11?ie=UTF8&node=2867979031&pf_rd_m=A3P5ROKL5A1OLE&pf_rd_s=merchandised-search-leftnav&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_t=101&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_i=66469031',
                                                   'hair_scalp_treatment':'https://www.amazon.co.uk/b/ref=amb_link_7?ie=UTF8&node=2867977031&pf_rd_m=A3P5ROKL5A1OLE&pf_rd_s=merchandised-search-leftnav&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_t=101&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_i=66469031',
                                                   'treatment_oil':'https://www.amazon.co.uk/hair-oil-argan/b/ref=amb_link_8?ie=UTF8&node=2867981031&pf_rd_m=A3P5ROKL5A1OLE&pf_rd_s=merchandised-search-leftnav&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_r=KF9SM53J2HXHP4EJD3AH&pf_rd_t=101&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_p=aaaa7182-fdd6-4b35-8f0b-993e78880b69&pf_rd_i=66469031'},
                                  'skin_care':{'body':{'cleanser':'https://www.amazon.co.uk/s/ref=lp_344269031_nr_n_3?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A344269031%2Cn%3A344282031&bbn=344269031&ie=UTF8&qid=1581612722&rnid=344269031',
                                                       'moisturizers':'https://www.amazon.co.uk/s/ref=lp_344269031_nr_n_1?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A344269031%2Cn%3A2805272031&bbn=344269031&ie=UTF8&qid=1581612722&rnid=344269031'},
                                               'eyes':{'creams':'https://www.amazon.co.uk/s/ref=lp_118465031_nr_n_0?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118465031%2Cn%3A344259031&bbn=118465031&ie=UTF8&qid=1581612984&rnid=118465031',
                                                       'gels':'https://www.amazon.co.uk/s/ref=lp_118465031_nr_n_1?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118465031%2Cn%3A344258031&bbn=118465031&ie=UTF8&qid=1581613044&rnid=118465031',
                                                       'serums':'https://www.amazon.co.uk/s/ref=lp_118465031_nr_n_3?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118465031%2Cn%3A344257031&bbn=118465031&ie=UTF8&qid=1581613044&rnid=118465031'},
                                               'face':{'cleansers':'https://www.amazon.co.uk/s/ref=lp_118466031_nr_n_1?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118466031%2Cn%3A344265031&bbn=118466031&ie=UTF8&qid=1581613120&rnid=118466031',
                                                       'moisturizers':'https://www.amazon.co.uk/s/ref=lp_118466031_nr_n_3?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118466031%2Cn%3A2805291031&bbn=118466031&ie=UTF8&qid=1581613120&rnid=118466031',
                                                       'toners':'https://www.amazon.co.uk/s/ref=lp_118466031_nr_n_0?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118466031%2Cn%3A344267031&bbn=118466031&ie=UTF8&qid=1581613120&rnid=118466031',
                                                       'treatments':'https://www.amazon.co.uk/s?bbn=118466031&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118466031%2Cn%3A18918424031&dc&fst=as%3Aoff&qid=1581613120&rnid=118466031&ref=lp_118466031_nr_n_7'},
                                               'lipcare':'https://www.amazon.co.uk/s/ref=lp_118464031_nr_n_4?fst=as%3Aoff&rh=n%3A117332031%2Cn%3A%21117333031%2Cn%3A118464031%2Cn%3A118467031&bbn=118464031&ie=UTF8&qid=1581613357&rnid=118464031'}},
             'food':{'tea':{'herbal':'https://www.amazon.co.uk/s?k=tea&i=grocery&rh=n%3A340834031%2Cn%3A358584031%2Cn%3A11711401%2Cn%3A406567031&dc&qid=1581613483&rnid=344155031&ref=sr_nr_n_1',
                            'green':'https://www.amazon.co.uk/s?k=tea&i=grocery&rh=n%3A340834031%2Cn%3A358584031%2Cn%3A11711401%2Cn%3A406566031&dc&qid=1581613483&rnid=344155031&ref=sr_nr_n_3',
                            'black':'https://www.amazon.co.uk/s?k=tea&i=grocery&rh=n%3A340834031%2Cn%3A358584031%2Cn%3A11711401%2Cn%3A406564031&dc&qid=1581613483&rnid=344155031&ref=sr_nr_n_2'},
                     'coffee':'https://www.amazon.co.uk/s?k=coffee&rh=n%3A340834031%2Cn%3A11711391&dc&qid=1581613715&rnid=1642204031&ref=sr_nr_n_2',
                     'dried_fruits':{'mixed':'https://www.amazon.co.uk/s?k=dried+fruits&rh=n%3A340834031%2Cn%3A9733163031&dc&qid=1581613770&rnid=1642204031&ref=sr_nr_n_2'},
                     'nuts':{'mixed':'https://www.amazon.co.uk/s?k=mixed&rh=n%3A359964031&ref=nb_sb_noss',
                             'peanuts':'https://www.amazon.co.uk/s?k=peanuts&rh=n%3A359964031&ref=nb_sb_noss',
                             'cashews':'https://www.amazon.co.uk/s?k=cashew&rh=n%3A359964031&ref=nb_sb_noss'}},
             'supplements':{'sports':{'pre_workout':'https://www.amazon.co.uk/b/?node=5977685031&ref_=Oct_s9_apbd_odnav_hd_bw_b35Hc3L_1&pf_rd_r=C5MZHH5TH5F868B6FQWD&pf_rd_p=8086b6c9-ae16-5c3c-a879-030afa4ee08f&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=2826478031',
                                      'protein':'https://www.amazon.co.uk/b/?node=2826510031&ref_=Oct_s9_apbd_odnav_hd_bw_b35Hc3L_0&pf_rd_r=C5MZHH5TH5F868B6FQWD&pf_rd_p=8086b6c9-ae16-5c3c-a879-030afa4ee08f&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=2826478031',
                                      'fat_burner':'https://www.amazon.co.uk/b/?node=5977737031&ref_=Oct_s9_apbd_odnav_hd_bw_b35Hc3L_2&pf_rd_r=C5MZHH5TH5F868B6FQWD&pf_rd_p=8086b6c9-ae16-5c3c-a879-030afa4ee08f&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=2826478031'},
                            'vitamins_dietary':{'supplements':'https://www.amazon.co.uk/b/?_encoding=UTF8&node=2826534031&bbn=65801031&ref_=Oct_s9_apbd_odnav_hd_bw_b35Hdc7_2&pf_rd_r=AY01DQVCB4SE7VVE7MTK&pf_rd_p=1ecdbf02-af23-502a-b7ab-9916ddd6690c&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=2826484031',
                                                'multivitamins':'https://www.amazon.co.uk/b/?_encoding=UTF8&node=2826506031&bbn=65801031&ref_=Oct_s9_apbd_odnav_hd_bw_b35Hdc7_1&pf_rd_r=AY01DQVCB4SE7VVE7MTK&pf_rd_p=1ecdbf02-af23-502a-b7ab-9916ddd6690c&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=2826484031'}},
             'wellness':{'massage_oil':'https://www.amazon.co.uk/b/?node=3360479031&ref_=Oct_s9_apbd_odnav_hd_bw_b50nmJ_4&pf_rd_r=GYVYF52HT2004EDTY67W&pf_rd_p=3f8e4361-c00b-588b-a07d-ff259bf98bbc&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=74073031',
                         'ayurveda':'https://www.amazon.co.uk/s?k=ayurveda&rh=n%3A65801031%2Cn%3A2826449031&dc&qid=1581686978&rnid=1642204031&ref=sr_nr_n_22'},
             'personal_accessories':{'bags':{'women':{'clutches':'https://www.amazon.co.uk/b/?node=1769563031&ref_=Oct_s9_apbd_odnav_hd_bw_b1vkt8h_3&pf_rd_r=VC8RX89R4V4JJ5TEBANF&pf_rd_p=cefca17f-8dac-5c80-848f-812aff1bfdd7&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=1769559031',
                                                      'crossbody':'https://www.amazon.co.uk/b/?node=1769564031&ref_=Oct_s9_apbd_odnav_hd_bw_b1vkt8h_1&pf_rd_r=VC8RX89R4V4JJ5TEBANF&pf_rd_p=cefca17f-8dac-5c80-848f-812aff1bfdd7&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=1769559031',
                                                      'fashion':'https://www.amazon.co.uk/b/?node=1769560031&ref_=Oct_s9_apbd_odnav_hd_bw_b1vkt8h_5&pf_rd_r=VC8RX89R4V4JJ5TEBANF&pf_rd_p=cefca17f-8dac-5c80-848f-812aff1bfdd7&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=1769559031',
                                                      'hobo':'https://www.amazon.co.uk/b/?node=1769565031&ref_=Oct_s9_apbd_odnav_hd_bw_b1vkt8h_4&pf_rd_r=VC8RX89R4V4JJ5TEBANF&pf_rd_p=cefca17f-8dac-5c80-848f-812aff1bfdd7&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=1769559031'}},
                                     'jewelry':{'anklets':'https://www.amazon.co.uk/s/ref=lp_10382835031_nr_n_0?fst=as%3Aoff&rh=n%3A193716031%2Cn%3A%21193717031%2Cn%3A10382835031%2Cn%3A10382860031&bbn=10382835031&ie=UTF8&qid=1581687575&rnid=10382835031',
                                                'bracelets':'https://www.amazon.co.uk/s/ref=lp_10382835031_nr_n_1?fst=as%3Aoff&rh=n%3A193716031%2Cn%3A%21193717031%2Cn%3A10382835031%2Cn%3A10382861031&bbn=10382835031&ie=UTF8&qid=1581687575&rnid=10382835031',
                                                'earrings':'https://www.amazon.co.uk/s/ref=lp_10382835031_nr_n_4?fst=as%3Aoff&rh=n%3A193716031%2Cn%3A%21193717031%2Cn%3A10382835031%2Cn%3A10382865031&bbn=10382835031&ie=UTF8&qid=1581687575&rnid=10382835031',
                                                'necklaces':'https://www.amazon.co.uk/s/ref=lp_10382835031_nr_n_7?fst=as%3Aoff&rh=n%3A193716031%2Cn%3A%21193717031%2Cn%3A10382835031%2Cn%3A10382868031&bbn=10382835031&ie=UTF8&qid=1581687575&rnid=10382835031',
                                                'rings':'https://www.amazon.co.uk/s/ref=lp_10382835031_nr_n_10?fst=as%3Aoff&rh=n%3A193716031%2Cn%3A%21193717031%2Cn%3A10382835031%2Cn%3A10382871031&bbn=10382835031&ie=UTF8&qid=1581687575&rnid=10382835031'},
                                     'artisan_fabrics':'https://www.amazon.co.uk/s?k=fabric&rh=n%3A11052681%2Cn%3A3063518031&dc&qid=1581687726&rnid=1642204031&ref=a9_sc_1'}}
amazon_india = {'health_and_beauty':{'hair_products':{'shampoo':'https://www.amazon.in/b/ref=s9_acss_bw_cg_btyH1_2a1_w?ie=UTF8&node=1374334031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-5&pf_rd_r=JHDJ4QHM0APVS05NGF4G&pf_rd_t=101&pf_rd_p=41b9c06b-1514-47de-a1c6-f4f13fb55ffe&pf_rd_i=1374305031',
                                                      'conditioner':'https://www.amazon.in/b/ref=s9_acss_bw_cg_btyH1_2b1_w?ie=UTF8&node=1374306031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-5&pf_rd_r=CBABMCW6C69JRBGZNWWP&pf_rd_t=101&pf_rd_p=41b9c06b-1514-47de-a1c6-f4f13fb55ffe&pf_rd_i=1374305031',
                                                      'treatment_oil':'https://www.amazon.in/b/ref=s9_acss_bw_cg_btyH1_2d1_w?ie=UTF8&node=3507139031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-5&pf_rd_r=ZDZ7D78V3T26TEDHJNG9&pf_rd_t=101&pf_rd_p=41b9c06b-1514-47de-a1c6-f4f13fb55ffe&pf_rd_i=1374305031',
                                                      'hair_loss':'https://www.amazon.in/b/ref=s9_acss_bw_cg_btyH1_4c1_w?ie=UTF8&node=1374317031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-5&pf_rd_r=ZDZ7D78V3T26TEDHJNG9&pf_rd_t=101&pf_rd_p=41b9c06b-1514-47de-a1c6-f4f13fb55ffe&pf_rd_i=1374305031',
                                                      'hair_scalp_treatment':'https://www.amazon.in/b/ref=s9_acss_bw_cg_btyH1_4d1_w?ie=UTF8&node=1374307031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-5&pf_rd_r=ZDZ7D78V3T26TEDHJNG9&pf_rd_t=101&pf_rd_p=41b9c06b-1514-47de-a1c6-f4f13fb55ffe&pf_rd_i=1374305031'},
                                     'skin_care':{'body':{'cleanser':'https://www.amazon.in/s/ref=lp_1374408031_nr_n_0?fst=as%3Aoff&rh=n%3A1355016031%2Cn%3A%211355017031%2Cn%3A1374407031%2Cn%3A1374408031%2Cn%3A1374276031&bbn=1374408031&ie=UTF8&qid=1584297005&rnid=1374408031',
                                                          'moisturizers':'https://www.amazon.in/s/ref=lp_1374408031_nr_n_2?fst=as%3Aoff&rh=n%3A1355016031%2Cn%3A%211355017031%2Cn%3A1374407031%2Cn%3A1374408031%2Cn%3A9530421031&bbn=1374408031&ie=UTF8&qid=1584297005&rnid=1374408031',
                                                          'treatments':'https://www.amazon.in/s?k=body+treatment&rh=n%3A1374408031&ref=nb_sb_noss'},
                                                  'face':{'f_cleansers':'https://www.amazon.in/b/?node=1374415031&ref_=Oct_CateC_1374414031_1&pf_rd_p=cb790f78-c066-5c5e-979e-6a9e1cf4426d&pf_rd_s=merchandised-search-3&pf_rd_t=101&pf_rd_i=1374414031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=6RP3WDZAHAN76CZ03SP8&pf_rd_r=6RP3WDZAHAN76CZ03SP8&pf_rd_p=cb790f78-c066-5c5e-979e-6a9e1cf4426d',
                                                          'f_moisturizers':'https://www.amazon.in/b/?node=9530421031&ref_=Oct_CateC_1374414031_2&pf_rd_p=cb790f78-c066-5c5e-979e-6a9e1cf4426d&pf_rd_s=merchandised-search-3&pf_rd_t=101&pf_rd_i=1374414031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=6RP3WDZAHAN76CZ03SP8&pf_rd_r=6RP3WDZAHAN76CZ03SP8&pf_rd_p=cb790f78-c066-5c5e-979e-6a9e1cf4426d',
                                                          'scrubs':'https://www.amazon.in/b/?node=1374426031&ref_=Oct_CateC_1374414031_4&pf_rd_p=cb790f78-c066-5c5e-979e-6a9e1cf4426d&pf_rd_s=merchandised-search-3&pf_rd_t=101&pf_rd_i=1374414031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=6RP3WDZAHAN76CZ03SP8&pf_rd_r=6RP3WDZAHAN76CZ03SP8&pf_rd_p=cb790f78-c066-5c5e-979e-6a9e1cf4426d',
                                                          'toners':'https://www.amazon.in/s/ref=lp_1374414031_nr_n_2?fst=as%3Aoff&rh=n%3A1355016031%2Cn%3A%211355017031%2Cn%3A1374407031%2Cn%3A1374414031%2Cn%3A1374429031&bbn=1374414031&ie=UTF8&qid=1584297008&rnid=1374414031'},
                                                  'eyes':{'creams':'https://www.amazon.in/b/?node=9530415031&ref_=Oct_CateC_1374413031_1&pf_rd_p=78c33add-7e83-5a69-ba19-ed52b4e40e91&pf_rd_s=merchandised-search-3&pf_rd_t=101&pf_rd_i=1374413031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=7Z12V8A4ZKY5VFWDAX67&pf_rd_r=7Z12V8A4ZKY5VFWDAX67&pf_rd_p=78c33add-7e83-5a69-ba19-ed52b4e40e91',
                                                          'gels':'https://www.amazon.in/b/?node=9530416031&ref_=Oct_CateC_1374413031_2&pf_rd_p=78c33add-7e83-5a69-ba19-ed52b4e40e91&pf_rd_s=merchandised-search-3&pf_rd_t=101&pf_rd_i=1374413031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=7Z12V8A4ZKY5VFWDAX67&pf_rd_r=7Z12V8A4ZKY5VFWDAX67&pf_rd_p=78c33add-7e83-5a69-ba19-ed52b4e40e91',
                                                          'serums':'https://www.amazon.in/b/?node=9530417031&ref_=Oct_CateC_1374413031_4&pf_rd_p=78c33add-7e83-5a69-ba19-ed52b4e40e91&pf_rd_s=merchandised-search-3&pf_rd_t=101&pf_rd_i=1374413031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=7Z12V8A4ZKY5VFWDAX67&pf_rd_r=7Z12V8A4ZKY5VFWDAX67&pf_rd_p=78c33add-7e83-5a69-ba19-ed52b4e40e91'},
                                                  'lipcare':'https://www.amazon.in/s/ref=lp_1374407031_nr_n_2?fst=as%3Aoff&rh=n%3A1355016031%2Cn%3A%211355017031%2Cn%3A1374407031%2Cn%3A1374438031&bbn=1374407031&ie=UTF8&qid=1584296218&rnid=1374407031'}},
              'food':{'tea':{'herbal':'https://www.amazon.in/s?k=tea&i=grocery&rh=n%3A2454178031%2Cn%3A4859478031%2Cn%3A4860063031%2Cn%3A4861893031&dc&qid=1584297639&rnid=2454179031&ref=sr_nr_n_2',
                             'green':'https://www.amazon.in/s?k=tea&i=grocery&rh=n%3A2454178031%2Cn%3A4859478031%2Cn%3A4860063031%2Cn%3A4861892031&dc&qid=1584297665&rnid=2454179031&ref=sr_nr_n_5',
                             'black':'https://www.amazon.in/s?k=tea&i=grocery&rh=n%3A2454178031%2Cn%3A4861889031&dc&qid=1584297583&rnid=2454179031&ref=sr_nr_n_2',
                             'chai':'https://www.amazon.in/s?k=tea&i=grocery&rh=n%3A2454178031%2Cn%3A4859478031%2Cn%3A4860063031%2Cn%3A14907955031&dc&qid=1584297691&rnid=2454179031&ref=sr_nr_n_3'},
                      'coffee':'https://www.amazon.in/s?k=tea&i=grocery&rh=n%3A2454178031%2Cn%3A4859478031%2Cn%3A4860057031&dc&qid=1584297737&rnid=2454179031&ref=sr_nr_n_5',
                      'dried_fruits':{'mixed':'https://www.amazon.in/s?k=dried+fruits&rh=n%3A2454178031%2Cn%3A4860222031&dc&qid=1584297781&rnid=3576079031&ref=sr_nr_n_2',
                                      'mangoes':'https://www.amazon.in/s?k=dried+fruits&i=grocery&rh=n%3A2454178031%2Cn%3A4859481031%2Cn%3A4859612031%2Cn%3A4860221031&dc&qid=1584297849&rnid=2454179031&ref=sr_nr_n_8'},
                      'nuts':{'mixed':'https://www.amazon.in/s?k=nuts&rh=n%3A2454178031%2Cn%3A4860240031&dc&qid=1584297938&rnid=3576079031&ref=sr_nr_n_2',
                              'peanuts':'https://www.amazon.in/s?k=nuts&rh=n%3A2454178031%2Cn%3A4860242031&dc&qid=1584297938&rnid=3576079031&ref=sr_nr_n_6',
                              'cashews':'https://www.amazon.in/s?k=nuts&rh=n%3A2454178031%2Cn%3A4860232031&dc&qid=1584297938&rnid=3576079031&ref=sr_nr_n_4'}},
              'supplements':{'sports':{'pre_workout':'https://www.amazon.in/s/ref=lp_1374491031_nr_n_1?fst=as%3Aoff&rh=n%3A1350384031%2Cn%3A%211350385031%2Cn%3A1374489031%2Cn%3A1374491031%2Cn%3A11364613031&bbn=1374491031&ie=UTF8&qid=1584298070&rnid=1374491031',
                                       'protein':'https://www.amazon.in/s/ref=lp_1374491031_nr_n_5?fst=as%3Aoff&rh=n%3A1350384031%2Cn%3A%211350385031%2Cn%3A1374489031%2Cn%3A1374491031%2Cn%3A11364592031&bbn=1374491031&ie=UTF8&qid=1584298070&rnid=1374491031',
                                       'fat_burner':'https://www.amazon.in/s/ref=lp_1374491031_nr_n_2?fst=as%3Aoff&rh=n%3A1350384031%2Cn%3A%211350385031%2Cn%3A1374489031%2Cn%3A1374491031%2Cn%3A11364684031&bbn=1374491031&ie=UTF8&qid=1584298070&rnid=1374491031',
                                       'weight_gainer':'https://www.amazon.in/s?bbn=1374491031&rh=n%3A1350384031%2Cn%3A%211350385031%2Cn%3A1374489031%2Cn%3A1374491031%2Cn%3A14095357031&dc&fst=as%3Aoff&qid=1584298070&rnid=1374491031&ref=lp_1374491031_nr_n_3'},
                             'vitamins_dietary':{'supplements':'https://www.amazon.in/b/ref=s9_acss_bw_cg_sbc_2c1_w?ie=UTF8&node=11364812031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-2&pf_rd_r=8B5XXC2N3HSMY7KBW9MJ&pf_rd_t=101&pf_rd_p=0720c767-867f-4b08-bfde-d4480dbf7639&pf_rd_i=1374492031',
                                                 'multivitamins':'https://www.amazon.in/b/ref=s9_acss_bw_cg_sbc_2b1_w?ie=UTF8&node=11364815031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-2&pf_rd_r=8B5XXC2N3HSMY7KBW9MJ&pf_rd_t=101&pf_rd_p=0720c767-867f-4b08-bfde-d4480dbf7639&pf_rd_i=1374492031'}},
              'wellness':{'ayurveda':'https://www.amazon.in/b/ref=s9_acss_bw_cg_HPC_3d1_w?ie=UTF8&node=4264020031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-2&pf_rd_r=BHZ3KGMYWFNKYYR07JAW&pf_rd_t=101&pf_rd_p=056a905c-f795-41f6-9489-198146d5ad1e&pf_rd_i=1350384031',
                          'essential_oil_set':'https://www.amazon.in/s?k=essential+oil+set&i=hpc&crid=2DS4QKSJUK6Z2&sprefix=essential+oil+%2Chpc%2C363&ref=nb_sb_ss_i_4_14',
                          'massage_oil':'https://www.amazon.in/s?k=massage+oil&i=hpc&rh=n%3A1350384031%2Cn%3A5745215031&dc&qid=1584298410&rnid=1350385031&swrs=6247812B73DAFDE28E02149D4063D7AA&ref=sr_nr_n_2'},
              'personal_accessories':{'bags':{'women':{'clutches':'https://www.amazon.in/s/ref=lp_1983338031_nr_n_0?fst=as%3Aoff&rh=n%3A1571283031%2Cn%3A%211571284031%2Cn%3A1983338031%2Cn%3A1983350031&bbn=1983338031&ie=UTF8&qid=1584298453&rnid=1983338031',
                                                       'crossbody':'https://www.amazon.in/s/ref=lp_1983338031_nr_n_4?fst=as%3Aoff&rh=n%3A1571283031%2Cn%3A%211571284031%2Cn%3A1983338031%2Cn%3A1983351031&bbn=1983338031&ie=UTF8&qid=1584298453&rnid=1983338031',
                                                       'hobo':'https://www.amazon.in/s?k=hobo&rh=n%3A1983338031&ref=nb_sb_noss'}},
                                      'jewelry':{'anklets':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454897011&dc&',
                                                 'bracelets':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454898011&dc&',
                                                 'earrings':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454917011&dc&',
                                                 'necklaces':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454917011&dc&',
                                                 'rings':'https://www.amazon.com/s?i=fashion-womens-intl-ship&bbn=16225018011&rh=n%3A16225018011%2Cn%3A7192394011%2Cn%3A7454939011&dc&'},
                                      'artisan_fabrics':'https://www.amazon.com/s?k=fabrics&rh=n%3A2617941011%2Cn%3A12899121&dc&'}}

amazon_aus = {'health_and_beauty':{'hair_products':{'shampoo':'https://www.amazon.com.au/b/?_encoding=UTF8&node=5150253051&bbn=4851917051&ref_=Oct_s9_apbd_odnav_hd_bw_b5cXATz&pf_rd_r=6SEM7GFDN7CQ2W4KXM9M&pf_rd_p=9dd4b462-1094-5e36-890d-bb1b694c8b53&pf_rd_s=merchandised-search-12&pf_rd_t=BROWSE&pf_rd_i=5150070051',
                                                    'conditioner':'https://www.amazon.com.au/b/?_encoding=UTF8&node=5150226051&bbn=4851917051&ref_=Oct_s9_apbd_odnav_hd_bw_b5cXATz&pf_rd_r=6SEM7GFDN7CQ2W4KXM9M&pf_rd_p=9dd4b462-1094-5e36-890d-bb1b694c8b53&pf_rd_s=merchandised-search-12&pf_rd_t=BROWSE&pf_rd_i=5150070051'},
                                     'skin_care':[],
                                     'wellness_product':[]},
                'food':{'tea':[],
                        'coffee':[],
                        'dried_fruits':[],
                        'nuts':[],
                        'supplements':[]},
                'personal_accessories':{'bags':[],
                                        'jewelry':[],
                                        'artisan_fabrics':[]}}

amazon = {'USA':amazon_usa,
          'UK':amazon_uk,
          'India':amazon_india,
          'Australia':amazon_aus}

### Execution

#### Getting all the product links
If all the product links are scraped and saved in the pickle file already, then don't run the *products_links* function

In [23]:
# #Initializing the product per Jupyter Notebook
country = 'USA'
cat1 = 'health_and_beauty'
cat2='skin_care'
# cat3='None'
# cat4 = 'None'
product='perfumes'
links,directory = products_links(country=country,category=cat1,cat2=cat2,product=product)

health_and_beauty/skin_care/perfumes
112
1
24
2
48
3
72
4
96
5
120
6
144
7
168
8
192
9
216
10
240
11
264
12
288
13
312
14
336
15
360
16
384
17
408
18
432
19
456
20
480
21
504
22
528
23
552
24
576
25
600
26
624
27
648
28
672
29
696
30
720
31
744
32
768
33
792
34
816
35
840
36
864
37
888
38
912
39
936
40
960
41
984
42
1008
43
1032
44
1056
45
1080
46
1104
47
1128
48
1152
49
1176
50
1200
51
1224
52
1248
53
1272
54
1296
55
1320
56
1344
57
1368
58
1392
59
1416
60
1440
61
1464
62
1488
63
1512
64
1536
65
1560
66
1584
67
1608
68
1632
69
1656
70
1680
71
1704
72
1728
73
1752
74
1776
75
1800
76
1824
77
1848
78
1872
79
1896
80
1920
81
1944
82
1968
83
1992
84
2016
85
2040
86
2064
87
2088
88
2112
89
2136
90
2160
91
2184
92
2208
93
2232
94
2256
95
2280
96
2304
97
2328
98
2352
99
2376
100
2400
101
2424
102
2448
103
2472
104
2496
105
2520
106
2544
107
2568
108
2592
109
2616
110
2640
111
2664
112
2688


In [24]:
# # Saving the scraped Links into a pickle file
# main = {'links':links,'directory':directory}
# with open('USA_Perfume.pkl', 'wb') as handle:
#     pickle.dump(main, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
#loading the pickle file
with open('USA_Perfume.pkl', 'rb') as f:
    file = pickle.load(f)
links = file['links']
directory = 'Amazon_USA/health_and_beauty/skin_care/perfumes'

In [109]:
def product_info(link,directory,directory_images,directory_html,country):
    '''Get all the product information of an Amazon Product

    PARAMETER
    ---------
    link: string
        Product link from the catalog of specific product
    directory: string
        Directory where all the data(images and database file) of the product
        will be stored
    country: string
        Country of interest

    RETURN
    ------
    product_title: string
        Title of the product
    rating_star: string
        Star rating of the product
    overall_rating: string
        Total number of feedbacks
    company: string
        Name of the company of the product
    price: string
        Price of the product based on country's currency
    product_highlights: string
        Short snippet of the product description
    product_length: string
        In terms of dimension, length of the product
    product_width: string
        In terms of dimension, width of the product
    product_height: string
        In terms of dimension, height of the product
    product_weight: string
        In terms of dimension, weight of the product
    asin: string
        Amazon Standard Identification Number, a unique identifier of each product in Amazon
    pd_unit: string
        Product Dimension SI unit
    best_seller_cat: string
        Best Seller rank of the product based on Category
    best_seller_prod: string
        Best Seller rank of the product based on Product type
    weight_unit: string
        SI unit of product weight
    shipping_weight: string
        Weight of the product including the shipping package
    shipping_weight_unit: string
        SI unit of shipping weight
    crr_5: string
        Customer Review Rating for 5 Stars
    crr_4: string
        Customer Review Rating for 4 Stars
    crr_3: string
        Customer Review Rating for 3 Stars
    crr_2: string
        Customer Review Rating for 2 Stars
    crr_1: string
        Customer Review Rating for 1 Stars
    crr_fr_1: string
        Customer Review Rating by feature for Feature 1
    crr_fr_2: string
        Customer Review Rating by feature for Feature 2
    crr_fr_3: string
        Customer Review Rating by feature for Feature 3
    tags: string
        List of tags associated with the product
    directory: string
        Directory in S3 bucket where the data(images and database file) is stored
    '''

    #Opening Selenium Webdrive with Amazon product
    driver = browser(link)
    time.sleep(4)
    scroll(driver)
    time.sleep(2)

    #Initializing BeautifulSoup operation in selenium browser
    selenium_soup = BeautifulSoup(driver.page_source, 'lxml')

    #Product Title
    try:
        product_title = driver.find_element_by_xpath('//*[@id="productTitle"]').text
    except:
        product_title = 'Not Scrapable'
    print(product_title)

    #Ratings - Star
    try:
        rating_star = float(selenium_soup.findAll('span',{'class':'a-icon-alt'})[0].text.split()[0])
    except:
        rating_star = 'Not Scrapable'
    print(rating_star)

    #Rating - Overall
    try:
        overall_rating = int(selenium_soup.findAll('span',{'id':'acrCustomerReviewText'})[0].text.split()[0].replace(',',''))
    except:
        overall_rating = 'Not Scrapable'
    print(overall_rating)

    #Company
    try:
        company = selenium_soup.findAll('a',{'id':'bylineInfo'})[0].text
    except:
        company = 'Not Scrapable'
    print(country)

    #Price
    try:
        denomination = '$'
        if country=='UAE':
            denomination = selenium_soup.findAll('span',{'id':'priceblock_ourprice'})[0].text[:3]
            price = float(selenium_soup.findAll('span',{'id':'priceblock_ourprice'})[0].text[3:])
        else:
            denomination = selenium_soup.findAll('span',{'id':'priceblock_ourprice'})[0].text[0]
            price = float(selenium_soup.findAll('span',{'id':'priceblock_ourprice'})[0].text[1:])
    except:
        try:
            if country=='UAE':
                try:
                    price = float(selenium_soup.findAll('span',{'id':'priceblock_ourprice'})[0].text[3:].replace(',',''))
                except:
                    price = float(selenium_soup.findAll('span',{'id':'priceblock_dealprice'})[0].text[3:].replace(',',''))
            else:
                try:
                    price = float(selenium_soup.findAll('span',{'id':'priceblock_ourprice'})[0].text[3:].replace(',',''))
                except:
                    price = float(selenium_soup.findAll('span',{'id':'priceblock_dealprice'})[0].text[3:].replace(',',''))
        except:
            denomination = 'Not Scrapable'
            price = 'Not Scrapable'
    print(denomination,price)

    #Product Highlights
    try:
        temp_ph = selenium_soup.findAll('div',{'id':'feature-bullets'})[0].findAll('li')
        counter_ph = len(temp_ph)
        product_highlights = []
        for i in range(counter_ph):
            raw = temp_ph[i].text
            clean = raw.strip()
            product_highlights.append(clean)
        product_highlights = '<CPT14>'.join(product_highlights)
    except:
        try:
            temp_ph = selenium_soup.findAll('div',{'id':'rich-product-description'})[0].findAll('p')
            counter_ph = len(temp_ph)
            product_highlights = []
            for i in range(counter_ph):
                raw = temp_ph[i].text
                clean = raw.strip()
                product_highlights.append(clean)
            product_highlights = '<CPT14>'.join(product_highlights)
        except:
            product_highlights = 'Not Available'
    print(product_highlights)

    #Helium

    #Product Details/Dimensions:
    #USA
    try:
        temp_pd = selenium_soup.findAll('div',{'id':'detailBulletsWrapper_feature_div'})[0].findAll('ul')[0].findAll('li')
        counter_pd = len(temp_pd)
        for i in range(counter_pd):
            try:
                if re.findall('ASIN',temp_pd[i].text.strip().split('\n')[0])[0]:
                    try:
                        asin = temp_pd[i].text.strip().split('\n')[3]
                    except:
                        pass
            except IndexError:
                pass
            try:
                if re.findall('Product Dimensions|Product Dimension|Product dimensions',temp_pd[i].text.strip().split('\n')[0])[0]:
                    pd_temp = temp_pd[i].text.strip().split('\n')[3].strip().split(';')
                    try:
                        product_length = float(pd_temp[0].split('x')[0])
                    except IndexError:
                        pass
                    try:
                        product_width = float(pd_temp[0].split('x')[1])
                    except IndexError:
                        pass
                    try:
                        product_height = float(pd_temp[0].split('x')[2].split(' ')[1])
                    except IndexError:
                        pass
                    try:
                        pd_unit = pd_temp[0].split('x')[2].split(' ')[2]
                    except IndexError:
                        pass
                    try:
                        product_weight = float(pd_temp[1].split(' ')[1])
                    except IndexError:
                        pass
                    try:
                        weight_unit = pd_temp[1].split(' ')[2]
                    except IndexError:
                        pass
            except:
                pass
            try:
                if re.findall('Shipping Weight|Shipping weight|shipping weight',temp_pd[i].text)[0]:
                    sweight_temp = temp_pd[i].text.split(':')[1].strip().split(' ')
                    shipping_weight = float(sweight_temp[0])
                    shipping_weight_unit = sweight_temp[1]
            except IndexError:
                pass
            try:
                if re.findall('Amazon Best Sellers Rank|Amazon Bestsellers Rank',temp_pd[i].text)[0]:
                    x = temp_pd[i].text.replace('\n','').split(' ')
                    indexes = []
                    for j,k in enumerate(x):
                        if re.findall('#',k):
                            indexes.append(j)
                    try:
                        best_seller_cat = int(temp_pd[i].text.strip().replace('\n','').split(' ')[3].replace(',',''))
                        best_seller_prod = int(x[indexes[0]].split('#')[1].split('in')[0])
                    except:
                        try:
                            best_seller_cat = x[indexes[0]].split('#')[1]
                        except:
                            pass
                        try:
                            best_seller_prod = x[indexes[1]].split('#')[1].split('in')[0]
                        except:
                            pass
            except IndexError:
                pass
        print(asin)
    except:
        pass
    try:
        temp_pd = selenium_soup.findAll('div',{'id':'detailBulletsWrapper_feature_div'})[1].findAll('ul')[0].findAll('li')
        counter_pd = len(temp_pd)
        for i in range(counter_pd):
            try:
                if re.findall('ASIN',temp_pd[i].text)[0]:
                    try:
                        asin = temp_pd[i].text.split(' ')[1]
                    except:
                        pass
            except IndexError:
                pass
            try:
                if re.findall('Product Dimensions|Product Dimension|Product dimensions',temp_pd[i].text)[0]:
                    pd_temp = temp_pd[i].text.strip().split('\n')[2].strip().split(';')
                    try:
                        product_length = float(pd_temp[0].split('x')[0])
                    except IndexError:
                        pass
                    try:
                        product_width = float(pd_temp[0].split('x')[1])
                    except IndexError:
                        pass
                    try:
                        product_height = float(pd_temp[0].split('x')[2].split(' ')[1])
                    except IndexError:
                        pass
                    try:
                        pd_unit = pd_temp[0].split('x')[2].split(' ')[2]
                    except IndexError:
                        pass
                    try:
                        product_weight = float(pd_temp[1].split(' ')[1])
                    except IndexError:
                        pass
                    try:
                        weight_unit = pd_temp[1].split(' ')[2]
                    except IndexError:
                        pass
            except:
                pass
            try:
                if re.findall('Shipping Weight|Shipping weight|shipping weight',temp_pd[i].text)[0]:
                    sweight_temp = temp_pd[i].text.split(':')[1].strip().split(' ')
                    shipping_weight = float(sweight_temp[0])
                    shipping_weight_unit = sweight_temp[1]
            except IndexError:
                pass
            try:
                if re.findall('Amazon Best Sellers Rank|Amazon Bestsellers Rank',temp_pd[i].text)[0]:
                    x = temp_pd[i].text.replace('\n','').split(' ')
                    indexes = []
                    for j,k in enumerate(x):
                        if re.findall('#',k):
                            indexes.append(j)
                    try:
                        best_seller_cat = int(temp_pd[i].text.strip().replace('\n','').split(' ')[3].replace(',',''))
                        best_seller_prod = int(x[indexes[0]].split('#')[1].split('in')[0])
                    except:
                        try:
                            best_seller_cat = x[indexes[0]].split('#')[1]
                        except:
                            pass
                        try:
                            best_seller_prod = x[indexes[1]].split('#')[1].split('in')[0]
                        except:
                            pass
            except IndexError:
                pass
        print(asin)
    except:
        pass

    #India
    try:
        temp_pd = selenium_soup.findAll('div',{'id':'detailBulletsWrapper_feature_div'})[0].findAll('ul')[0].findAll('li')
        counter_pd = len(temp_pd)
        for i in range(counter_pd):
            try:
                if re.findall('ASIN',temp_pd[i].text)[0]:
                    asin = temp_pd[i].text.split(' ')[1]
            except:
                pass
            try:
                if re.findall('Product Dimensions|Product Dimension|Product dimensions',temp_pd[i].text)[0]:
                    pd_temp = temp_pd[i].text.strip().split('\n')[2].strip().split(' ')
                    try:
                        product_length = float(pd_temp[0])
                    except:
                        pass
                    try:
                        product_width = float(pd_temp[2])
                    except:
                        pass
                    try:
                        product_height = float(pd_temp[4])
                    except:
                        pass
                    try:
                        pd_unit = pd_temp[5]
                    except:
                        pass
                    try:
                        product_weight = float(pd_temp[1].split(' ')[1])
                    except:
                        pass
                    try:
                        weight_unit = pd_temp[1].split(' ')[2]
                    except:
                        pass
                print(asin)
            except IndexError:
                pass
            try:
                if re.findall('Shipping Weight|Shipping weight|shipping weight',temp_pd[i].text)[0]:
                    sweight_temp = temp_pd[i].text.split(':')[1].strip().split(' ')
                    shipping_weight = float(sweight_temp[0])
                    shipping_weight_unit = sweight_temp[1]
            except IndexError:
                pass
            try:
                if re.findall('Item Weight|Product Weight|Item weight|Product weight|Boxed-product Weight',temp_pd[i].text)[0]:
                    pd_weight_temp = temp_pd[i].text.replace('\n','').strip().split('     ')[1].strip()
                    product_weight = float(pd_weight_temp.split(' ')[0])
                    weight_unit = pd_weight_temp.split(' ')[1]
            except IndexError:
                pass
            try:
                if re.findall('Amazon Best Sellers Rank|Amazon Bestsellers Rank',temp_pd[i].text)[0]:
                    x = temp_pd[i].text.strip().replace('\n','').split(' ')
                    indexes = []
                    for j,k in enumerate(x):
                        if re.findall('#',k):
                            indexes.append(j)
                    try:
                        best_seller_cat = int(temp_pd[i].text.strip().replace('\n','').split(' ')[3].replace(',',''))
                        best_seller_prod = int(x[indexes[0]].split('#')[1].split('in')[0])
                    except:
                        try:
                            best_seller_cat = x[indexes[0]].split('#')[1]
                        except:
                            pass
                        try:
                            best_seller_prod = x[indexes[1]].split('#')[1].split('in')[0]
                        except:
                            pass
            except IndexError:
                pass
            print(asin)
    except:
        pass
    try:
        try:
            asin = list(selenium_soup.findAll('div',{'class':'pdTab'})[1].findAll('tr')[0].findAll('td')[1])[0]
        except:
            pass
        try:
            dimensions = list(selenium_soup.findAll('div',{'class':'pdTab'})[0].findAll('tr')[0].findAll('td')[1])[0]
        except:
            pass
        try:
            weight_temp = list(selenium_soup.findAll('div',{'class':'pdTab'})[1].findAll('tr')[1].findAll('td')[1])[0]
        except:
            pass
        try:
            best_seller_cat = float(list(selenium_soup.findAll('div',{'class':'pdTab'})[1].findAll('tr')[5].findAll('td')[1])[0].split('\n')[-1].split(' ')[0].replace(',',''))
        except:
            pass
        try:
            best_seller_prod = int(list(list(list(list(selenium_soup.findAll('div',{'class':'pdTab'})[1].findAll('tr')[5].findAll('td')[1])[5])[1])[1])[0].replace('#',''))
        except:
            pass
        try:
            product_length = float(dimensions.split('x')[0])
        except:
            pass
        try:
            product_width = float(dimensions.split('x')[1])
        except:
            pass
        try:
            product_height = float(dimensions.split('x')[2].split(' ')[1])
        except:
            pass
        try:
            product_weight = weight_temp.split(' ')[0]
        except:
            pass
        try:
            weight_unit = weight_temp.split(' ')[1]
        except:
            pass
        try:
            pd_unit = dimensions.split(' ')[-1]
        except:
            pass
        print(asin)
    except:
        try:
            for j in [0,1]:
                temp_pd = selenium_soup.findAll('table',{'class':'a-keyvalue prodDetTable'})[j].findAll('tr')
                for i in range(len(temp_pd)):
                    if re.findall('ASIN',temp_pd[i].text):
                        asin = temp_pd[i].text.strip().split('\n')[3].strip()
                    if re.findall('Item Model Number|Item model number',temp_pd[i].text):
                        bait = temp_pd[i].text.strip().split('\n')[3].strip()
                    if re.findall('Best Sellers Rank|Amazon Best Sellers Rank|Amazon Bestsellers Rank',temp_pd[i].text):
                        x = temp_pd[i].text.strip().replace('\n','').split(' ')
                        indexes = []
                        for j,k in enumerate(x):
                            if re.findall('#',k):
                                indexes.append(j)
                        best_seller_cat = int(x[indexes[0]].split('#')[1])
                        best_seller_prod = int(x[indexes[1]].split('#')[1].split('in')[0])
                    if re.findall('Product Dimensions|Product dimension|Product Dimension',temp_pd[i].text):
                        dimensions = temp_pd[i].text.strip().split('\n')[3].strip().split('x')
                        product_length = float(dimensions[0].strip())
                        product_width = float(dimensions[1].strip())
                        product_height = float(dimensions[2].strip().split(' ')[0])
                        pd_unit = dimensions[2].strip().split(' ')[1]
                    if re.findall('Item Weight|Product Weight|Item weight|Boxed-product Weight',temp_pd[i].text):
                        weight_temp = temp_pd[i].text.strip().split('\n')[3].strip()
                        product_weight = float(weight_temp.split(' ')[0])
                        weight_unit = weight_temp.split(' ')[1]
                    if re.findall('Shipping Weight|Shipping weight|shipping weight',temp_pd[i].text):
                        sweight_temp = temp_pd[i].text.replace('\n','').strip().split('                      ')[1].lstrip().split(' ')
                        shipping_weight = float(sweight_temp[0])
                        shipping_weight_unit = sweight_temp[1]
                print(asin,bait)
        except:
            try:
                temp_pd = selenium_soup.findAll('div',{'id':'prodDetails'})[0].findAll('tr')
                for i in range(len(temp_pd)):
                    if re.findall('ASIN',temp_pd[i].text):
                        asin = temp_pd[i].text.strip().split('\n')[3].strip()
                    if re.findall('Best Sellers Rank|Amazon Best Sellers Rank|Amazon Bestsellers Rank',temp_pd[i].text):
                        x = temp_pd[i].text.strip().replace('\n','').split(' ')
                        indexes = []
                        for j,k in enumerate(x):
                            if re.findall('#',k):
                                indexes.append(j)
                        best_seller_cat = int(x[indexes[0]].split('#')[1])
                        best_seller_prod = int(x[indexes[1]].split('#')[1].split('in')[0])
                    if re.findall('Product Dimensions|Product dimension|Product Dimension',temp_pd[i].text):
                        dimensions = temp_pd[i].text.strip().split('\n')[3].strip().split('x')
                        product_length = float(dimensions[0].strip())
                        product_width = float(dimensions[1].strip())
                        product_height = float(dimensions[2].strip().split(' ')[0])
                        pd_unit = dimensions[2].strip().split(' ')[1]
                    if re.findall('Item Weight|Product Weight|Item weight|Boxed-product Weight',temp_pd[i].text):
                        weight_temp = temp_pd[i].text.strip().split('\n')[3].strip()
                        product_weight = float(weight_temp.split(' ')[0])
                        weight_unit = weight_temp.split(' ')[1]
                    if re.findall('Shipping Weight|Shipping weight|shipping weight',temp_pd[i].text):
                        sweight_temp = temp_pd[i].text.replace('\n','').strip().split('                      ')[1].lstrip().split(' ')
                        shipping_weight = float(sweight_temp[0])
                        shipping_weight_unit = sweight_temp[1]
            except:
                try:
                    temp_pd = selenium_soup.findAll('div',{'id':'detail_bullets_id'})[0].findAll('tr')[0].findAll('li')
                    for i in range(len(temp_pd)):
                        if re.findall('ASIN',temp_pd[i].text):
                            asin = temp_pd[i].text.strip().split(':')[1].strip()
                        if re.findall('Best Sellers Rank|Amazon Best Sellers Rank|Amazon Bestsellers Rank',temp_pd[i].text):
                            x = temp_pd[i].text.strip().replace('\n','').split(' ')
                            indexes = []
                            for j,k in enumerate(x):
                                if re.findall('#',k):
                                    indexes.append(j)
                            best_seller_cat = int(x[indexes[0]].split('#')[1])
                            best_seller_prod = int(x[indexes[1]].split('#')[1].split('in')[0])
                        if re.findall('Product Dimensions|Product dimension|Product Dimension',temp_pd[i].text):
                            dimensions = temp_pd[i].text.strip().split('\n')[2].strip().split('x')
                            product_length = float(dimensions[0].strip())
                            product_width = float(dimensions[1].strip())
                            product_height = float(dimensions[2].strip().split(' ')[0])
                            pd_unit = dimensions[2].strip().split(' ')[1]
                        if re.findall('Item Weight|Product Weight|Item weight|Boxed-product Weight',temp_pd[i].text):
                            weight_temp = temp_pd[i].text.strip().split('\n')[2].strip()
                            product_weight = float(weight_temp.split(' ')[0])
                            weight_unit = weight_temp.split(' ')[1]
                        if re.findall('Shipping Weight|Shipping weight|shipping weight',temp_pd[i].text):
                            sweight_temp = temp_pd[i].text.replace('\n','').strip().split('                      ')[1].lstrip().split(' ')
                            shipping_weight = float(sweight_temp[0])
                            shipping_weight_unit = sweight_temp[1]
                except:
                    pass
    try:
        print(asin)
    except NameError:
        asin = 'Not Scrapable'
    try:
        print(best_seller_cat)
    except NameError:
        best_seller_cat = 'Not Scrapable'
    try:
        print(best_seller_prod)
    except NameError:
        best_seller_prod = 'Not Scrapable'
    try:
        print(product_length)
    except NameError:
        product_length = 'Not Scrapable'
    try:
        print(product_width)
    except NameError:
        product_width = 'Not Scrapable'
    try:
        print(product_height)
    except NameError:
        product_height = 'Not Scrapable'
    try:
        print(product_weight)
    except NameError:
        product_weight = 'Not Scrapable'
    try:
        print(weight_unit)
    except NameError:
        weight_unit = 'Not Scrapable'
    try:
        print(pd_unit)
    except NameError:
        pd_unit = 'Not Scrapable'
    try:
        print(shipping_weight_unit)
    except NameError:
        shipping_weight_unit = 'Not Scrapable'
    try:
        print(shipping_weight)
    except NameError:
        shipping_weight = 'Not Scrapable'

    print('Product Length: {}, product_width: {}, product_height: {},product_weight: {}, asin: {}, pd_unit: {}, best_seller_cat: {}, best_seller_prod: {} , weight_unit: {},shipping_weight: {}, shipping_weight_unit: {}'.format(product_length,
        product_width,product_height,product_weight,asin,pd_unit,
        best_seller_cat,best_seller_prod,weight_unit,shipping_weight,
        shipping_weight_unit))

    #Customer Review Ratings - Overall
    time.sleep(0.5)
    try:
        temp_crr = selenium_soup.findAll('table',{'id':'histogramTable'})[1].findAll('a')
        crr_main = {}
        crr_temp = []
        counter_crr = len(temp_crr)
        for i in range(counter_crr):
            crr_temp.append(temp_crr[i]['title'])
        crr_temp = list(set(crr_temp))
        for j in range(len(crr_temp)):
            crr_temp[j] = crr_temp[j].split(' ')
            stopwords = ['stars','represent','of','rating','reviews','have']
            for word in list(crr_temp[j]):
                if word in stopwords:
                    crr_temp[j].remove(word)
            print(crr_temp[j])
            try:
                if re.findall(r'%',crr_temp[j][1])[0]:
                    crr_main.update({int(crr_temp[j][0]): int(crr_temp[j][1].replace('%',''))})
            except:
                crr_main.update({int(crr_temp[j][1]): int(crr_temp[j][0].replace('%',''))})
    except:
        try:
            temp_crr = selenium_soup.findAll('table',{'id':'histogramTable'})[1].findAll('span',{'class':'a-offscreen'})
            crr_main = {}
            counter_crr = len(temp_crr)
            star = counter_crr
            for i in range(counter_crr):
                crr_main.update({star:int(temp_crr[i].text.strip().split('/n')[0].split(' ')[0].replace('%',''))})
                star -= 1
        except:
            pass
    try:
        crr_5 = crr_main[5]
    except:
        crr_5 = 0
    try:
        crr_4 = crr_main[4]
    except:
        crr_4 = 0
    try:
        crr_3 = crr_main[3]
    except:
        crr_3 = 0
    try:
        crr_2 = crr_main[2]
    except:
        crr_2 = 0
    try:
        crr_1 = crr_main[1]
    except:
        crr_1 = 0

    #Customer Review Ratings - By Feature
    time.sleep(1)
    try:
        driver.find_element_by_xpath('//*[@id="cr-summarization-attributes-list"]/div[4]/a/span').click()
        temp_fr = driver.find_element_by_xpath('//*[@id="cr-summarization-attributes-list"]').text
        temp_fr = temp_fr.split('\n')
        crr_feature_title = []
        crr_feature_rating = []
        for i in [0,2,4]:
             crr_feature_title.append(temp_fr[i])
        for j in [1,3,5]:
            crr_feature_rating.append(temp_fr[j])
        crr_feature = dict(zip(crr_feature_title,crr_feature_rating))
    except:
        try:
            temp_fr = driver.find_element_by_xpath('//*[@id="cr-summarization-attributes-list"]').text
            temp_fr = temp_fr.split('\n')
            crr_feature_title = []
            crr_feature_rating = []
            for i in [0,2,4]:
                 crr_feature_title.append(temp_fr[i])
            for j in [1,3,5]:
                crr_feature_rating.append(temp_fr[j])
            crr_feature = dict(zip(crr_feature_title,crr_feature_rating))
        except:
            crr_feature = 'Not Defined'
    try:
        crr_feature_key = list(crr_feature.keys())
    except:
        pass
    try:
        crr_fr_1 = crr_feature[crr_feature_key[0]]
    except:
        crr_fr_1 = 0
    try:
        crr_fr_2 = crr_feature[crr_feature_key[1]]
    except:
        crr_fr_2 = 0
    try:
        crr_fr_3 = crr_feature[crr_feature_key[2]]
    except:
        crr_fr_3 = 0

    #Tags:
    time.sleep(1)
    try:
        temp_tags = selenium_soup.findAll('div',{'class':'cr-lighthouse-terms'})[0]
        counter_tags = len(temp_tags)
        print('Counter Tags:',counter_tags)
        tags = []
        for i in range(counter_tags):
            tags.append(temp_tags.findAll('span')[i].text.strip())
            print(tags[i])
    except:
        tags = ['None']
    try:
        for feature in crr_feature_key:
            tags.append(feature)
    except:
        pass
    tags = list(set(tags))
    tags = '<CPT14>'.join(tags)
    print(tags)

    build_dir(directory_images)
    os.chdir(directory_images)
    #Images
    images = []
    for i in [0,3,4,5,6,7,8,9]:
        try:
            images.append(selenium_soup.findAll('div',{'class':'imgTagWrapper'})[i].find('img')['src'])
        except:
            pass
    import urllib.request
    for i  in range(len(images)):
        if asin =='Not Scrapable':
            product_image = "{}_{}.jpg".format(product_title,i)
            product_image = product_image.replace('/','')
            urllib.request.urlretrieve(images[i],product_image)
#             upload_s3("{}_{}.jpg".format(product_title,i),
#                       directory+"/images/" + product_image)
#             delete_images(product_image)
        else:
            product_image = "{}_{}.jpg".format(asin,i)
            product_image = product_image.replace('/','')
            urllib.request.urlretrieve(images[i],product_image)
#             upload_s3("{}_{}.jpg".format(asin,i),
#                       directory+"/images/" + product_image)
#             delete_images(product_image)
    build_dir(directory_html)
    os.chdir(directory_html)
    try:
        with open("{}.html".format(asin), "w",encoding='utf-8') as file:
            file.write(str(selenium_soup))
    except:
        try:
            with open("{}.html".format(product_title), "w",encoding='utf-8') as file:
                file.write(str(selenium_soup))
        except:
            with open("{}.html".format(link), "w",encoding='utf-8') as file:
                file.write(str(selenium_soup))
    build_dir(directory)
    os.chdir(directory)

    return [product_title,rating_star,overall_rating,company,price,
            product_highlights,product_length,product_width,product_height,
            product_weight,asin,pd_unit,best_seller_cat,best_seller_prod,
            weight_unit,shipping_weight,shipping_weight_unit,crr_5,crr_4,
            crr_3,crr_2,crr_1,crr_fr_1,crr_fr_2,crr_fr_3,tags,directory]

In [108]:
directory = r"C:\Users\Dell-pc\Documents\GitHub\Product-Packaging\Perfumes-Data"
directory_images = r"C:\Users\Dell-pc\Documents\GitHub\Product-Packaging\Perfumes-Data\images"
directory_html = r"C:\Users\Dell-pc\Documents\GitHub\Product-Packaging\Perfumes-Data\html"

In [42]:
## Initializing the checkpoint if the product scraping has already been initialized in the past
# Run before and after an interruption
new_links = checkpoint(links,directory,product)

In [107]:
def build_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)
build_dir(r"C:\Users\Dell-pc\Documents\GitHub\Product-Packaging\test")

In [96]:
#replace links with new_links if interruption
for link in new_links:
    print(link)
    data = product_info(link=link,directory=directory,country=country)
    conn = sqlite3.connect('{}.db'.format(product))
    database(product_data=data,link=link,country=country,
             cat1=cat1,cat2=cat2,product=product)

https://www.amazon.com/Jo-Malone-Cologne-TRAVEL-spray/dp/B017F309UK/ref=sr_1_121?dchild=1&qid=1606408929&s=beauty-intl-ship&sr=1-121
Jo Malone Wood Sage & Sea Salt Cologne 0.3 oz / 9 ml TRAVEL spray (0.3 oz)
4.4
319
USA
$ 30.0

B017F309UK
B017F309UK
B017F309UK
B017F309UK
B017F309UK
B017F309UK
B017F309UK
B017F309UK
0.75
0.75
2.5
1.28
Ounces
inches
Product Length: 0.75, product_width: 0.75, product_height: 2.5,product_weight: 1.28, asin: B017F309UK, pd_unit: inches, best_seller_cat: Not Scrapable, best_seller_prod: Not Scrapable , weight_unit: Ounces,shipping_weight: Not Scrapable, shipping_weight_unit: Not Scrapable
['1', '4%']
['4', '15%']
['5', '70%']
['2', '2%']
['3', '9%']
Counter Tags: 17
last long
last long
small bottle
small bottle
love this scent
love this scent
scent so much
scent so much
fragrance love
fragrance love
scent i love
scent i love
malone
malone
smell
smell
perfume
Thickness<CPT14>Scent<CPT14>scent so much<CPT14>perfume<CPT14>malone<CPT14>fragrance love<CPT14>For tr

A bold, invigorating blend of sage, bergamot spice & leather woods
B07MYWCPY8
B07MYWCPY8
B07MYWCPY8
B07MYWCPY8
B07MYWCPY8
B07MYWCPY8
B07MYWCPY8
B07MYWCPY8
Product Length: Not Scrapable, product_width: Not Scrapable, product_height: Not Scrapable,product_weight: Not Scrapable, asin: B07MYWCPY8, pd_unit: Not Scrapable, best_seller_cat: Not Scrapable, best_seller_prod: Not Scrapable , weight_unit: Not Scrapable,shipping_weight: Not Scrapable, shipping_weight_unit: Not Scrapable
['1', '1%']
['5', '83%']
['4', '12%']
['3', '4%']
Counter Tags: 7
bath and body
bath and body
body works
body works
smells great
smells great
scent
scent<CPT14>body works<CPT14>bath and body<CPT14>smells great
https://www.amazon.com/Estee-Lauder-Beyond-Paradise-Parfum/dp/B00U3GKI9E/ref=sr_1_128?dchild=1&qid=1606408929&s=beauty-intl-ship&sr=1-128
Estee Lauder Beyond Paradise Eau De Parfum for Women, 1.7 Ounce
4.7
217
USA
Not Scrapable Not Scrapable
The package dimension of the product is 14.7cmL x 10.2cmW x 5.1cmH
B

smells just like
smells just like
last long
last long
waste your money
waste your money
like abercrombie
like abercrombie
smells great
smells great
exactly like
exactly like
smells amazing
smells amazing
blaze cologne
blaze cologne
smelling cologne
smelling cologne
abercrombie fierce
like fierce<CPT14>like abercrombie<CPT14>Scent<CPT14>Longevity<CPT14>abercrombie fierce<CPT14>exactly like<CPT14>blaze cologne<CPT14>Value for money<CPT14>smells amazing<CPT14>last long<CPT14>smelling cologne<CPT14>smells just like<CPT14>waste your money<CPT14>smells great
https://www.amazon.com/Cruz-Rose-Water-Spray-Made/dp/B00270AG7S/ref=sr_1_134?dchild=1&qid=1606408929&s=beauty-intl-ship&sr=1-134
De La Cruz Rose Water Spray, No Parabens or Artificial Colors, Vegan, Made in USA 8 FL. OZ.
4.4
351
USA
$ 5.94
Made in USA<CPT14>No Parabens or Artificial Colors<CPT14>Not Tested on Animals<CPT14>Vegan
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S
B00270AG7S


fresh scent<CPT14>sport strike<CPT14>great scent<CPT14>smells amazing<CPT14>last long<CPT14>gym bag<CPT14>mens fragrance<CPT14>eau de toilette<CPT14>clean fresh<CPT14>couple of hours<CPT14>easy to use
https://www.amazon.com/WIND-SONG-BODY-SPRAY-PACKAGE/dp/B00KBYQDI8/ref=sr_1_140?dchild=1&qid=1606408929&s=beauty-intl-ship&sr=1-140
WIND SONG BODY SPRAY - PACKAGE OF 3
4.7
349
USA
$ 19.49
WIND SONG BODY SPRAY<CPT14>PRINCE MATCHABELLI<CPT14>CLASSIC,ELEGANT AND FEMININE<CPT14>TIME-HONORED AND RESILIENT<CPT14>PACKAGE OF 3 - 2.5 OZ. EACH
B00KBYQDI8
B00KBYQDI8
B00KBYQDI8
B00KBYQDI8
B00KBYQDI8
B00KBYQDI8
B00KBYQDI8
Product Length: Not Scrapable, product_width: Not Scrapable, product_height: Not Scrapable,product_weight: Not Scrapable, asin: B00KBYQDI8, pd_unit: Not Scrapable, best_seller_cat: Not Scrapable, best_seller_prod: Not Scrapable , weight_unit: Not Scrapable,shipping_weight: Not Scrapable, shipping_weight_unit: Not Scrapable
['5', '83%']
['3', '5%']
['1', '2%']
['2', '1%']
['4', '10%']


LAYALI Perfume Oil for Women 15mL | Delicate Florals with sultry Black Currant, Plum, Ylang, Rose, Jasmine and Amber | Natural Alcohol Free Attar | Exotic Body Oil by Artisan Swiss Arabian Oud
4.3
865
USA
Not Scrapable Not Scrapable
Mouthwatering: white florals and fruity sweet fragrance has a slight woody infusion blend for a noir extreme experience.<CPT14>Sexy: an Eau de Parfum concentrated perfume oil for women that is irresistibly seductive and unique. Modern, intimate and feminine, like a second skin.<CPT14>Performance: Long to very long lasting with great sillage rating; Alcohol free and all natural and oriental Oudh perfume (Agarwood).<CPT14>This is a Concentrated Perfume Oil (CPO), do not smell it straight out of the bottle; It will smell very strong - apply to your wrist and allow to settle for 2 to 3 minutes before passing judgement. When you can't smell it anymore, ask a trusted friend if they still can. Olfactive fatigue (also known as nose blindness) is typical of concentr

...

['4', '14%']
['5', '62%']
['2', '7%']
['3', '9%']
Counter Tags: 21
smells like
smells like
blazing saddles
blazing saddles
solid colognes
solid colognes
fire in the hole
fire in the hole
last long
last long
new car
new car
great scent
great scent
long time
long time
air freshener
air freshener
never used
never used
long lasting
Giftable<CPT14>never used<CPT14>fire in the hole<CPT14>long lasting<CPT14>blazing saddles<CPT14>great scent<CPT14>Longevity<CPT14>Scent<CPT14>smells like<CPT14>solid colognes<CPT14>long time<CPT14>last long<CPT14>new car<CPT14>air freshener
https://www.amazon.com/Bath-Body-Works-CHERRY-Fragrance/dp/B07QNZKBKF/ref=sr_1_213?dchild=1&qid=1606408955&s=beauty-intl-ship&sr=1-213
Bath and Body Works BLACK CHERRY MERLOT Fine Fragrance Mist 8 Fluid Ounce (2019 Edition, White Barn Label)
4.5
284
USA
$ 17.99
FRAGRANCE: A rich, fruity blend of dark cherry, black raspberry & sumptuous merlot<CPT14>Lavishly splash or lightly spritz, either way you'll fall in love at first mis

Thickness<CPT14>Scent<CPT14>brands and the scent<CPT14>smell amazing<CPT14>highly recommend<CPT14>Value for money<CPT14>scent ever<CPT14>yankee candle<CPT14>bergamot and amber<CPT14>favorite scent<CPT14>smell like<CPT14>candle ever<CPT14>years ago<CPT14>masculine scent
https://www.amazon.com/Michael-Kors-White-Parfum-Spray/dp/B00SC0ZYRE/ref=sr_1_219?dchild=1&qid=1606408973&s=beauty-intl-ship&sr=1-219
Michael Kors White Eau de Parfum Spray for Women, 3.4 Ounce
4.7
181
USA
$ 55.59
Launched by the design house of Michael Kors in the year 2014<CPT14>It has a floral fragrance<CPT14>ylang-ylang, tuberose white peony, gardenia, jasmine, white musk, and amber notes
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
B00SC0ZYRE
3.2
1.7
4.12
0.16
Ounces
inches
Product Length: 3.2, product_width: 1.7, product_height: 4.12,product_weight: 0.16, asin: B00SC0ZYRE, pd_unit: inches, best_seller_cat: Not Scrapable, best_seller_prod: Not Scrapable , weight_unit: Ounce

https://www.amazon.com/Comptoir-Sud-Pacifique-Vanille-Toilette/dp/B005C66P7E/ref=sr_1_226?dchild=1&qid=1606408973&s=beauty-intl-ship&sr=1-226
Comptoir Sud Pacifique Vanille Extreme Eau de Toilette, 1 Fl Oz
4.5
187
USA
$ 42.0
Not Available
B005C66P7E
B005C66P7E
B005C66P7E
B005C66P7E
B005C66P7E
B005C66P7E
B005C66P7E
B005C66P7E
B005C66P7E
Product Length: Not Scrapable, product_width: Not Scrapable, product_height: Not Scrapable,product_weight: Not Scrapable, asin: B005C66P7E, pd_unit: Not Scrapable, best_seller_cat: Not Scrapable, best_seller_prod: Not Scrapable , weight_unit: Not Scrapable,shipping_weight: Not Scrapable, shipping_weight_unit: Not Scrapable
['3', '6%']
['4', '6%']
['2', '2%']
['5', '79%']
['1', '7%']
Counter Tags: 21
smells like
smells like
sud pacifique
sud pacifique
cotton candy
cotton candy
get compliments
get compliments
favorite perfume
favorite perfume
sugar cookies
sugar cookies
like vanilla
like vanilla
every time
every time
vanilla cupcake
vanilla cupcake
comptoi

https://www.amazon.com/ESCENTRIC-MOLECULES-Toilette-Travel-Molecule/dp/B006MK6178/ref=sr_1_232?dchild=1&qid=1606408973&s=beauty-intl-ship&sr=1-232
Escentric Molecules Eau de Toilette Spray Travel Case Molecule 01, 1.05 Fl Oz
4.0
143
USA
$ 80.0
Not Available
B006MK6178
B006MK6178
B006MK6178
B006MK6178
B006MK6178
B006MK6178
B006MK6178
B006MK6178
B006MK6178
3.94
4.33
4.72
1.05
Ounces
inches
Product Length: 3.94, product_width: 4.33, product_height: 4.72,product_weight: 1.05, asin: B006MK6178, pd_unit: inches, best_seller_cat: Not Scrapable, best_seller_prod: Not Scrapable , weight_unit: Ounces,shipping_weight: Not Scrapable, shipping_weight_unit: Not Scrapable
['5', '67%']
['3', '7%']
['2', '6%']
['1', '15%']
['4', '5%']
Counter Tags: 6
love this perfume
love this perfume
fake
fake
scent
scent
love this perfume<CPT14>fake<CPT14>scent
https://www.amazon.com/Carolina-Herrera-Parfum-Spray-Sample/dp/B07GDBCDTR/ref=sr_1_233?dchild=1&qid=1606408973&s=beauty-intl-ship&sr=1-233
3 Carolina Herrera

TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.939
  (Session info: headless chrome=87.0.4280.66)


In [97]:
len(new_links)

2568

In [98]:
# Run if there is an interruption
new_links = checkpoint(links,directory,product)

In [99]:
len(new_links)

2454